# 📊 Evaluate Model (While Training)

Download output từ kernel đang train để evaluate.

---

In [ ]:
# 1. Setup Kaggle API credentials
# Get your API key from: https://www.kaggle.com/settings → API → Create New Token
# This downloads kaggle.json

import os
os.makedirs('/root/.config/kaggle', exist_ok=True)

# REPLACE with your credentials from kaggle.json!
KAGGLE_USERNAME = "YOUR_USERNAME"
KAGGLE_KEY = "YOUR_API_KEY"

with open('/root/.config/kaggle/kaggle.json', 'w') as f:
    f.write(f'{{"username":"{KAGGLE_USERNAME}","key":"{KAGGLE_KEY}"}}')

os.chmod('/root/.config/kaggle/kaggle.json', 0o600)
print("✓ Kaggle API configured")

In [ ]:
# 2. Download output từ kernel training
!kaggle kernels output kiengioisk/nlp-project -p /kaggle/working/output
print("\n✓ Download complete")

In [ ]:
# 3. Check downloaded files
!ls -la /kaggle/working/output/
!echo "\n=== Checkpoints ==="
!ls -la /kaggle/working/output/checkpoints/ 2>/dev/null

In [ ]:
# 4. Install deps & setup
!pip install -q sentencepiece sacrebleu python-dotenv tqdm pyyaml

# Clone repo for code
!rm -rf /kaggle/working/repo
!git clone https://github.com/TranKien2005/EV_Translate_Modle_NLP_Project.git /kaggle/working/repo
%cd /kaggle/working/repo

In [ ]:
# 5. Find checkpoint and tokenizer paths
from pathlib import Path

output = Path('/kaggle/working/output')

# Find files
ckpts = list(output.rglob('*.pt'))
toks = list(output.rglob('*.model'))

print("Checkpoints found:")
for c in ckpts:
    print(f"  {c}")

print("\nTokenizers found:")
for t in toks:
    print(f"  {t}")

# Set paths
CKPT = str(ckpts[0]) if ckpts else None
TOK_SRC = str([t for t in toks if 'src' in t.name][0]) if toks else None
TOK_TGT = str([t for t in toks if 'tgt' in t.name][0]) if toks else None

print(f"\nUsing: {CKPT}")

In [ ]:
# 6. Load model
from src.evaluate import load_translator

translator = load_translator(
    checkpoint_path=CKPT,
    vocab_src_path=TOK_SRC,
    vocab_tgt_path=TOK_TGT
)
print("✓ Model loaded!")

In [ ]:
# 7. Test translations
for s in ["Hello", "How are you?", "Thank you", "I love you", "Good morning"]:
    print(f"EN: {s}")
    print(f"VI: {translator.translate(s)}")
    print()